In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print('cell 1 DONE')

cell 1 DONE


In [2]:
import pydicom
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf

2021-10-10 16:50:39.142940: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
data_dir = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"
patients = os.listdir(data_dir)
labels_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
label=labels_df.loc[:,'MGMT_value']

scan = ["FLAIR","T1w","T1wCE","T2w"]
labels_df.head()

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1


In [4]:
n=100

In [5]:
i=0
for patient in patients[:]:
    
    path = data_dir + '/'+ patient + "/" + scan[0]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    # print(len(slices),label[i],slices[i].pixel_array.shape)
    i+=1
    print(i)
print("Done")
print(len(slices))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
print(slices)

[Dataset.file_meta -------------------------------
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['DERIVED', 'SECONDARY']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.8.498.82220772065687065683632979760412180022
(0008, 0050) Accession Number                    SH: '00049'
(0008, 0060) Modality                            CS: 'MR'
(0008, 103e) Series Description                  LO: 'FLAIR'
(0010, 0010) Patient's Name                      PN: '00049'
(0010, 0020) Patient ID                          LO: '00049'
(0018, 0023) MR Acquisition Type                 CS: '3D'
(0018, 0050) Slice Thickness                     DS: '1.4'
(0018, 0081) Echo Time                           DS: None
(0018, 0082

In [7]:
print(np.array(slices).shape)

(296,)


In [8]:
np.array(slices[0].pixel_array)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [9]:
#FUNCTIONS WE USE:
l=[1,2,3,4,5,6,7,8,9]
def chunk(l,n):
    for i in range(0,len(l),n):
        yield l[i:i+n]

# print(list(chunk(l,3)))       
        
def mean(l):
    return sum(l)/len(l)

def resize(layer,HM_slices):
    if(len(layer)<HM_slices and HM_slices-len(layer)>1):
        layer.append(layer[-1])
        return resize(layer,HM_slices)
    elif(len(layer)<HM_slices and HM_slices-len(layer)==1):
        layer.append(layer[-1])
        return layer
    else:
        return layer
    
    
    
def process_data(patients, label_df, IMG_PX_SIZE = 50,HM_slices=20, visual=False, i=0):
    path = data_dir + '/'+ patient + "/" + scan[scan_type]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    i+=1

       
    new_slices = []

    slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

    chunk_sizes =  math.ceil(len(slices)/HM_slices)

    for slice_chunk in chunk(slices, chunk_sizes):
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)

    new_slices=resize(new_slices,HM_slices)
    len(new_slices)
        
    if visual:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice,cmap='gray')
        #         y.imshow(each_slice)
        plt.show()
            
    if label[i]==1: label[i]=np.array([0,1])
    elif label[i]==0: label[i]=np.array([1,0])

    return np.array(new_slices), label [i]

In [10]:
IMG_PX_SIZE = 32
HM_slices=20
i=0
j=0 
much_data=[]
ps=[]
for patient in patients[:]:
    # ps=[]
    for scan_type in range(1):
        
        path = data_dir + '/'+ patient + "/" + scan[scan_type]
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    #     print(len(slices),label[i],slices[i].pixel_array.shape)
        i+=1

       
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        print(slices[0].shape)

        chunk_sizes =  math.ceil(len(slices)/HM_slices)
    #     print(chunk_sizes)

        for slice_chunk in chunk(slices, chunk_sizes):
    #         print(slice_chunk)
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)
        
        # new_slices=np.array(new_slices)

        # print(type(np.array(new_slices)))
        # print(new_slices.shape)

        new_slices=resize(new_slices,HM_slices)
#         print(len(new_slices))
        
        # if label[i]==1: label[i]=[0,1]
        # elif label[i]==0: label[i]=[1,0]


    ps.append(new_slices)
    new_slices=np.array(new_slices)
#     Tlabel=np.array(label[0:10])
    label=np.array(label)
    print(new_slices.shape)
    print(patient)


    # much_data.append(ps)
# plt.show()
# print('I am done')
# print(len(much_data),len(ps))
print("done")

(32, 32)
(20, 32, 32)
00688
(32, 32)
(20, 32, 32)
00058
(32, 32)
(20, 32, 32)
00516
(32, 32)
(20, 32, 32)
00185
(32, 32)
(20, 32, 32)
00547
(32, 32)
(20, 32, 32)
00502
(32, 32)
(20, 32, 32)
00222
(32, 32)
(20, 32, 32)
00610
(32, 32)
(20, 32, 32)
00133
(32, 32)
(20, 32, 32)
00452
(32, 32)
(20, 32, 32)
00199
(32, 32)
(20, 32, 32)
00364
(32, 32)
(20, 32, 32)
00680
(32, 32)
(20, 32, 32)
00421
(32, 32)
(20, 32, 32)
00694
(32, 32)
(20, 32, 32)
00159
(32, 32)
(20, 32, 32)
00683
(32, 32)
(20, 32, 32)
00454
(32, 32)
(20, 32, 32)
00346
(32, 32)
(20, 32, 32)
00816
(32, 32)
(20, 32, 32)
00708
(32, 32)
(20, 32, 32)
00286
(32, 32)
(20, 32, 32)
00406
(32, 32)
(20, 32, 32)
00094
(32, 32)
(20, 32, 32)
00158
(32, 32)
(20, 32, 32)
00416
(32, 32)
(20, 32, 32)
00517
(32, 32)
(20, 32, 32)
00576
(32, 32)
(20, 32, 32)
00728
(32, 32)
(20, 32, 32)
00359
(32, 32)
(20, 32, 32)
00136
(32, 32)
(20, 32, 32)
00139
(32, 32)
(20, 32, 32)
00389
(32, 32)
(20, 32, 32)
00305
(32, 32)
(20, 32, 32)
00624
(32, 32)
(20, 32, 32

In [11]:
ps=np.array(ps)
ps = np.expand_dims(ps, axis=-1)
ps.shape

(585, 20, 32, 32, 1)

In [12]:
# sample_shape = (32, 32, 20, 1)
#MODEL
model = Sequential()
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(20,32,32,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Flatten())
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


2021-10-10 17:08:46.202210: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-10 17:08:46.257047: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-10 17:08:46.314940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-10 17:08:46.315647: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-10 17:08:46.315778: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-10 17:08:46.346550: I tensorflow/stream_executor/platform/def

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 30, 30, 64)    1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 15, 15, 64)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 15, 15, 64)     256       
_________________________________________________________________
dropout (Dropout)            (None, 9, 15, 15, 64)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 7, 13, 13, 32)     55328     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 6, 6, 32)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 6, 6, 32)       1

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label = np.array(label[:])
label_encode_x = OneHotEncoder(sparse=False)
label=label.reshape(585,1)
label=label_encode_x.fit_transform(label)

In [14]:
# len(new_slices)
label

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [15]:
# Fit data to model
model.fit(x=ps, y=label, batch_size=1,validation_split=0.3,epochs=50,verbose=1)

2021-10-10 17:08:50.580500: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 33505280 exceeds 10% of free system memory.
2021-10-10 17:08:50.666267: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-10 17:08:50.677932: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000189999 Hz


Epoch 1/50


2021-10-10 17:08:51.396864: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-10 17:08:52.174897: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-10 17:08:52.192473: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


409/409 [==============================] - 9s 6ms/step - loss: 1.2816 - accuracy: 0.4900 - val_loss: 0.8632 - val_accuracy: 0.5511
Epoch 2/50
409/409 [==============================] - 2s 5ms/step - loss: 1.1742 - accuracy: 0.5469 - val_loss: 0.9333 - val_accuracy: 0.5170
Epoch 3/50
409/409 [==============================] - 2s 5ms/step - loss: 0.9679 - accuracy: 0.6129 - val_loss: 0.8801 - val_accuracy: 0.5057
Epoch 4/50
409/409 [==============================] - 2s 5ms/step - loss: 1.1103 - accuracy: 0.5445 - val_loss: 0.8418 - val_accuracy: 0.5511
Epoch 5/50
409/409 [==============================] - 2s 5ms/step - loss: 1.0469 - accuracy: 0.5289 - val_loss: 0.8676 - val_accuracy: 0.4943
Epoch 6/50
409/409 [==============================] - 2s 5ms/step - loss: 0.9960 - accuracy: 0.5309 - val_loss: 0.8722 - val_accuracy: 0.5170
Epoch 7/50
409/409 [==============================] - 2s 5ms/step - loss: 0.9097 - accuracy: 0.6112 - val_loss: 0.7395 - val_accuracy: 0.5114
Epoch 8/50
409/40

In [16]:
IMG_PX_SIZE = 32
HM_slices=20
data_dir = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"
patients = os.listdir(data_dir)
i=0
j=0 
much_data=[]
Testps=[]
for patient in patients[:]:
    # ps=[]
    for scan_type in range(1):
        
        path = data_dir + '/'+ patient + "/" + scan[scan_type]
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    #     print(len(slices),label[i],slices[i].pixel_array.shape)
        i+=1

       
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        print(slices[0].shape)

        chunk_sizes =  math.ceil(len(slices)/HM_slices)
    #     print(chunk_sizes)

        for slice_chunk in chunk(slices, chunk_sizes):
    #         print(slice_chunk)
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)
        
        # new_slices=np.array(new_slices)

        # print(type(np.array(new_slices)))
        # print(new_slices.shape)

        new_slices=resize(new_slices,HM_slices)
#         print(len(new_slices))
        
        # if label[i]==1: label[i]=[0,1]
        # elif label[i]==0: label[i]=[1,0]


    Testps.append(new_slices)
    new_slices=np.array(new_slices)
    Tlabel=np.array(label[0:10])
    print(new_slices.shape)
    print(patient)
print("DONE")

(32, 32)
(20, 32, 32)
00114
(32, 32)
(20, 32, 32)
00013
(32, 32)
(20, 32, 32)
00821
(32, 32)
(20, 32, 32)
00644
(32, 32)
(20, 32, 32)
00699
(32, 32)
(20, 32, 32)
00125
(32, 32)
(20, 32, 32)
00833
(32, 32)
(20, 32, 32)
00762
(32, 32)
(20, 32, 32)
00393
(32, 32)
(20, 32, 32)
00287
(32, 32)
(20, 32, 32)
00434
(32, 32)
(20, 32, 32)
00337
(32, 32)
(20, 32, 32)
00145
(32, 32)
(20, 32, 32)
00307
(32, 32)
(20, 32, 32)
00489
(32, 32)
(20, 32, 32)
00749
(32, 32)
(20, 32, 32)
00163
(32, 32)
(20, 32, 32)
00384
(32, 32)
(20, 32, 32)
00438
(32, 32)
(20, 32, 32)
00047
(32, 32)
(20, 32, 32)
00323
(32, 32)
(20, 32, 32)
00428
(32, 32)
(20, 32, 32)
00462
(32, 32)
(20, 32, 32)
00560
(32, 32)
(20, 32, 32)
00422
(32, 32)
(20, 32, 32)
00825
(32, 32)
(20, 32, 32)
00335
(32, 32)
(20, 32, 32)
00082
(32, 32)
(20, 32, 32)
00458
(32, 32)
(20, 32, 32)
01006
(32, 32)
(20, 32, 32)
00573
(32, 32)
(20, 32, 32)
00647
(32, 32)
(20, 32, 32)
00603
(32, 32)
(20, 32, 32)
00712
(32, 32)
(20, 32, 32)
00181
(32, 32)
(20, 32, 32

In [17]:
Testps=np.array(Testps)
Testps = np.expand_dims(Testps, axis=-1)
Testps.shape

(87, 20, 32, 32, 1)

In [18]:
predictions = model.predict(x=Testps,verbose=0)

In [19]:
# np.savetxt('submission.csv',predictions, delimiter=',') 

In [20]:
# %ls "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

In [21]:
len(predictions)

87

In [22]:
data_dir = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"
tPatients = os.listdir(data_dir)

In [23]:
tPatients

['00114',
 '00013',
 '00821',
 '00644',
 '00699',
 '00125',
 '00833',
 '00762',
 '00393',
 '00287',
 '00434',
 '00337',
 '00145',
 '00307',
 '00489',
 '00749',
 '00163',
 '00384',
 '00438',
 '00047',
 '00323',
 '00428',
 '00462',
 '00560',
 '00422',
 '00825',
 '00335',
 '00082',
 '00458',
 '01006',
 '00573',
 '00647',
 '00603',
 '00712',
 '00181',
 '00200',
 '00129',
 '00829',
 '00535',
 '00721',
 '00037',
 '00015',
 '00463',
 '00447',
 '00662',
 '00492',
 '00702',
 '00027',
 '00381',
 '00681',
 '00592',
 '00208',
 '00213',
 '00521',
 '00585',
 '00355',
 '00001',
 '00091',
 '00190',
 '00826',
 '00229',
 '00135',
 '00719',
 '00553',
 '00769',
 '00460',
 '00503',
 '00822',
 '00372',
 '00779',
 '00997',
 '00333',
 '00079',
 '00256',
 '00450',
 '00467',
 '00595',
 '00671',
 '00252',
 '00153',
 '00161',
 '00264',
 '00474',
 '00174',
 '00119',
 '00080',
 '00182']

In [24]:
# for i in range(len(tPatients)):
#     tPatients[i]=int(tPatients[i])
# tPatients

In [25]:
l=[]
for i in range(len(tPatients)):
    l.append(np.argmax(predictions[i]))

In [26]:
final=list(zip(tPatients,l))

In [27]:
final

[('00114', 1),
 ('00013', 1),
 ('00821', 0),
 ('00644', 1),
 ('00699', 0),
 ('00125', 0),
 ('00833', 0),
 ('00762', 0),
 ('00393', 0),
 ('00287', 0),
 ('00434', 0),
 ('00337', 0),
 ('00145', 0),
 ('00307', 0),
 ('00489', 1),
 ('00749', 1),
 ('00163', 0),
 ('00384', 0),
 ('00438', 0),
 ('00047', 0),
 ('00323', 0),
 ('00428', 1),
 ('00462', 1),
 ('00560', 1),
 ('00422', 0),
 ('00825', 1),
 ('00335', 0),
 ('00082', 1),
 ('00458', 1),
 ('01006', 1),
 ('00573', 1),
 ('00647', 1),
 ('00603', 1),
 ('00712', 1),
 ('00181', 0),
 ('00200', 0),
 ('00129', 0),
 ('00829', 0),
 ('00535', 1),
 ('00721', 1),
 ('00037', 0),
 ('00015', 1),
 ('00463', 1),
 ('00447', 1),
 ('00662', 0),
 ('00492', 0),
 ('00702', 0),
 ('00027', 0),
 ('00381', 1),
 ('00681', 1),
 ('00592', 1),
 ('00208', 0),
 ('00213', 0),
 ('00521', 1),
 ('00585', 0),
 ('00355', 0),
 ('00001', 0),
 ('00091', 1),
 ('00190', 1),
 ('00826', 0),
 ('00229', 1),
 ('00135', 1),
 ('00719', 0),
 ('00553', 1),
 ('00769', 0),
 ('00460', 1),
 ('00503',

In [28]:
s=list((('BraTS21ID','MGMT_value'),))

In [29]:
for i in range(len(final)):
    s.append(final[i])

In [30]:
s

[('BraTS21ID', 'MGMT_value'),
 ('00114', 1),
 ('00013', 1),
 ('00821', 0),
 ('00644', 1),
 ('00699', 0),
 ('00125', 0),
 ('00833', 0),
 ('00762', 0),
 ('00393', 0),
 ('00287', 0),
 ('00434', 0),
 ('00337', 0),
 ('00145', 0),
 ('00307', 0),
 ('00489', 1),
 ('00749', 1),
 ('00163', 0),
 ('00384', 0),
 ('00438', 0),
 ('00047', 0),
 ('00323', 0),
 ('00428', 1),
 ('00462', 1),
 ('00560', 1),
 ('00422', 0),
 ('00825', 1),
 ('00335', 0),
 ('00082', 1),
 ('00458', 1),
 ('01006', 1),
 ('00573', 1),
 ('00647', 1),
 ('00603', 1),
 ('00712', 1),
 ('00181', 0),
 ('00200', 0),
 ('00129', 0),
 ('00829', 0),
 ('00535', 1),
 ('00721', 1),
 ('00037', 0),
 ('00015', 1),
 ('00463', 1),
 ('00447', 1),
 ('00662', 0),
 ('00492', 0),
 ('00702', 0),
 ('00027', 0),
 ('00381', 1),
 ('00681', 1),
 ('00592', 1),
 ('00208', 0),
 ('00213', 0),
 ('00521', 1),
 ('00585', 0),
 ('00355', 0),
 ('00001', 0),
 ('00091', 1),
 ('00190', 1),
 ('00826', 0),
 ('00229', 1),
 ('00135', 1),
 ('00719', 0),
 ('00553', 1),
 ('00769',

In [31]:
# for i in range(len(tPatients)):
#     final[i]=list(final[i])

In [32]:
# final=np.array(final)

In [33]:
np.savetxt("submission.csv",s ,delimiter=',',fmt='%s')

In [34]:
# predictions

In [35]:
# data.to_csv('file1.csv')